In [11]:
# -*- coding: utf-8 -*-
"""
Capstone Report Pipeline (single-file) - COMPLETE VERSION
Author: Jeevan Deep Borugadda

Implements ALL steps described in the report including SMOTE, XGBoost weighting, and modern macro integration.

Run:
    python capstone_report_pipeline.py

Requires:
    numpy, pandas, scikit-learn, matplotlib, xgboost, imbalanced-learn
"""

# ==== Imports & config ====
import os
import math
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_fscore_support,
    roc_curve, precision_recall_curve, confusion_matrix, brier_score_loss
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import calibration_curve

# Imbalanced learning for SMOTE
try:
    from imblearn.over_sampling import SMOTE, RandomOverSampler
    from imblearn.pipeline import Pipeline as ImbPipeline
    HAS_IMBLEARN = True
except Exception:
    HAS_IMBLEARN = False

# XGBoost
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# ==== Report-aligned constants ====
RAW_CSV_PATH = "bank-additional-full.csv"  # must be present in working dir
OUTDIR = Path("capstone_outputs_complete")  # all artifacts land here

CAT_COLS = [
    'job','marital','education','default','housing','loan',
    'contact','month','day_of_week','poutcome'
]
DROP_LEAKAGE = ['duration']
DROP_REDUNDANT = ['nr.employed']  # highly collinear with euribor3m; keep euribor3m
SKEWED_NUMS = ['campaign','previous','pdays_non999']  # winsorize 1/99
TOP_FRACTION = 0.10  # business operating point (top 10% calling budget)

# Modern macroeconomic values (illustrative placeholders for 2023–2025 scenario testing)
MODERN_MACROS = {
    'euribor3m': 3.5,
    'cons.conf.idx': -20.0,
    'cons.price.idx': 105.0,
    'emp.var.rate': 0.5
}


# -----------------------------------------------------------------------------
#                            Data Preparation
# -----------------------------------------------------------------------------
def group_rare(series: pd.Series, min_frac: float = 0.005, preserve: List[str] = ['unknown']) -> pd.Series:
    s = series.astype(str).str.lower()
    freqs = s.value_counts(normalize=True)
    rare = [c for c, f in freqs.items() if f < min_frac and c not in preserve]
    return s.replace({c: 'Other' for c in rare})

def winsorize_clip(s: pd.Series, low_q=0.01, high_q=0.99) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    lo, hi = s.quantile(low_q), s.quantile(high_q)
    return s.clip(lo, hi)

def clean_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """
    Implements report Section 4:
    - target y (0/1)
    - Drop leakage (duration)
    - pdays sentinel → contacted_before, pdays_non999
    - Rare-level grouping for categories; keep 'unknown'
    - Winsorization for skewed numerics
    - Drop redundant macro (nr.employed)
    """
    df = df.copy()

    # Target
    df['y'] = (df['y'].astype(str).str.lower() == 'yes').astype(int)

    # Drop leakage
    for c in DROP_LEAKAGE:
        if c in df.columns:
            df.drop(columns=c, inplace=True, errors='ignore')

    # Sentinel handling
    df['contacted_before'] = (df['pdays'] != 999).astype(int)
    df['pdays_non999'] = df['pdays'].replace(999, np.nan)

    # Rare-level grouping for listed categoricals
    for c in CAT_COLS:
        if c in df.columns:
            df[c] = group_rare(df[c], min_frac=0.005, preserve=['unknown'])

    # Winsorize right-skewed numeric fields
    for c in SKEWED_NUMS:
        if c in df.columns:
            df[c] = winsorize_clip(df[c], 0.01, 0.99)

    # Drop redundant macro
    for c in DROP_REDUNDANT:
        if c in df.columns:
            df.drop(columns=c, inplace=True, errors='ignore')

    return df

# -----------------------------------------------------------------------------
#                            Preprocessors & Models
# -----------------------------------------------------------------------------
def build_preprocessor(X: pd.DataFrame, algo: str) -> ColumnTransformer:
    """
    Family-specific preprocessing per report:
    - LR  : numeric median-impute + StandardScaler; pdays_non999 -> -1 + scale; OHE cats
    - RF  : numeric median-impute, NO scaling; pdays_non999 -> -1; OHE cats
    - XGB : numeric passthrough (keep NaNs so tree can split on missing), NO scaling; OHE cats
    """
    num_all = X.select_dtypes(include=[np.number]).columns.tolist()
    special = "pdays_non999"
    other_nums = [c for c in num_all if c != special]

    # --- Categorical encoder
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
    cat_cols_present = [c for c in CAT_COLS if c in X.columns]
    cat_tf = Pipeline([("ohe", ohe)])

    transformers = []

    if algo == "lr":
        # Other numerics: median + scale
        if other_nums:
            transformers.append(("num_other", Pipeline([
                ("imputer", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler())
            ]), other_nums))
        # pdays_non999: -1 + scale
        if special in X.columns:
            transformers.append(("num_pdays", Pipeline([
                ("imputer", SimpleImputer(strategy="constant", fill_value=-1)),
                ("scaler", StandardScaler())
            ]), [special]))

    elif algo == "rf":
        # Other numerics: median ONLY (no scaling)
        if other_nums:
            transformers.append(("num_other", Pipeline([
                ("imputer", SimpleImputer(strategy="median")),
            ]), other_nums))
        # pdays_non999: -1 (no scaling)
        if special in X.columns:
            transformers.append(("num_pdays", Pipeline([
                ("imputer", SimpleImputer(strategy="constant", fill_value=-1)),
            ]), [special]))

    elif algo == "xgb":
        # Let XGB learn missing splits: leave numerics as-is (passthrough)
        if other_nums:
            transformers.append(("num_other", "passthrough", other_nums))
        if special in X.columns:
            transformers.append(("num_pdays", "passthrough", [special]))
    else:
        raise ValueError("algo must be 'lr' | 'rf' | 'xgb'")

    if cat_cols_present:
        transformers.append(("cat", cat_tf, cat_cols_present))

    return ColumnTransformer(transformers=transformers, remainder="drop")

def make_model(algo: str, use_resampling: str = None):
    if algo == "lr":
        # With ROS we do not also use class_weight
        if use_resampling == "ros":
            return LogisticRegression(max_iter=500, random_state=RANDOM_SEED)
        else:
            return LogisticRegression(max_iter=500, class_weight="balanced", random_state=RANDOM_SEED)

    
    if algo == "rf":
        if use_resampling == "ros":
            return RandomForestClassifier(
                n_estimators=500, max_depth=None, min_samples_leaf=2,
                n_jobs=-1, random_state=RANDOM_SEED  # No class_weight with ROS
            )
        else:
            return RandomForestClassifier(
                n_estimators=500, max_depth=None, min_samples_leaf=2,
                class_weight="balanced", n_jobs=-1, random_state=RANDOM_SEED
            )
    
    if algo == "xgb":
        if not HAS_XGB:
            raise ImportError("xgboost is not installed. pip install xgboost")
        # Calculate scale_pos_weight for 1:8 imbalance (88.73% / 11.27% ≈ 7.87)
        scale_pos_weight = 7.87
        return XGBClassifier(
            n_estimators=500, learning_rate=0.05, max_depth=6,
            subsample=0.8, colsample_bytree=0.8, scale_pos_weight=scale_pos_weight,
            random_state=RANDOM_SEED, n_jobs=-1, tree_method="hist"
        )
    raise ValueError("algo must be 'lr' | 'rf' | 'xgb'")

def make_pipeline(X_train: pd.DataFrame, algo: str, use_resampling: str = None) -> Pipeline:
    pre = build_preprocessor(X_train, algo)

    if use_resampling and HAS_IMBLEARN:
        if use_resampling == "ros" and algo == "lr":
            # Random OverSampling for LR (safe with OHE)
            ros = RandomOverSampler(random_state=RANDOM_SEED)
            clf = make_model(algo, use_resampling)
            return ImbPipeline([
                ("pre", pre),
                ("ros", ros),
                ("clf", clf)
            ])
        elif use_resampling == "ros" and algo == "rf":
            # Random OverSampling for RF (no class_weight)
            ros = RandomOverSampler(random_state=RANDOM_SEED)
            clf = make_model(algo, use_resampling)
            return ImbPipeline([
                ("pre", pre),
                ("ros", ros),
                ("clf", clf)
            ])

    
    # Default pipeline without resampling
    clf = make_model(algo, use_resampling)
    return Pipeline([("pre", pre), ("clf", clf)])

# -----------------------------------------------------------------------------
#                            Metrics & Plots
# -----------------------------------------------------------------------------
def decile_metrics(y_true: np.ndarray, y_proba: np.ndarray, top_frac: float = TOP_FRACTION) -> Dict[str, float]:
    n = len(y_proba)
    k = max(1, int(np.floor(top_frac * n)))
    idx = np.argsort(-y_proba)[:k]
    captured_positives = int(y_true[idx].sum())
    total_positives = int(y_true.sum())
    capture_rate = (captured_positives / total_positives) if total_positives > 0 else 0.0
    lift = (capture_rate / top_frac) if top_frac > 0 else 0.0
    return {
        "decile_top_frac": top_frac,
        "decile_top_k": k,
        "decile_captured_positives": captured_positives,
        "decile_total_positives": total_positives,
        "decile_capture_rate": float(capture_rate),
        "decile_lift": float(lift),
        "decile_calls_saved_percent": float(1 - top_frac)
    }

def threshold_for_top_frac(y_proba: np.ndarray, top_frac: float = TOP_FRACTION) -> float:
    n = len(y_proba)
    k = max(1, int(np.floor(top_frac * n)))
    return float(np.sort(y_proba)[::-1][k-1])

def eval_classification(y_true: np.ndarray, y_proba: np.ndarray, threshold: float = 0.5) -> Dict[str, float]:
    y_pred = (y_proba >= threshold).astype(int)
    auc = roc_auc_score(y_true, y_proba)
    pr_auc = average_precision_score(y_true, y_proba)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    brier = brier_score_loss(y_true, y_proba)
    return {
        "auc": float(auc),
        "pr_auc": float(pr_auc),
        f"precision@{threshold}": float(prec),
        f"recall@{threshold}": float(rec),
        f"f1@{threshold}": float(f1),
        "brier_score": float(brier)
    }

def business_point_metrics(y_true: np.ndarray, y_proba: np.ndarray, top_frac: float = TOP_FRACTION) -> Dict[str, float]:
    thr = threshold_for_top_frac(y_proba, top_frac)
    base = eval_classification(y_true, y_proba, threshold=thr)
    deci = decile_metrics(y_true, y_proba, top_frac=top_frac)
    return {"threshold_at_top_frac": thr, **base, **deci}

def make_decile_table(y_true, y_proba, n_deciles: int = 10) -> pd.DataFrame:
    y_true = np.asarray(y_true).astype(int)
    y_proba = np.asarray(y_proba).astype(float)
    order = np.argsort(-y_proba)
    y_sorted = y_true[order]
    proba_sorted = y_proba[order]
    n = len(y_true)
    total_pos = y_true.sum()
    idx_splits = np.array_split(np.arange(n), n_deciles)
    rows = []
    for i, idxs in enumerate(idx_splits, start=1):
        pos_in_bin = int(y_sorted[idxs].sum())
        pop_frac = len(idxs)/n
        capture_in_bin = pos_in_bin/total_pos if total_pos>0 else 0.0
        lift = (capture_in_bin/pop_frac) if pop_frac>0 else 0.0
        rows.append({
            "decile": i,
            "n_in_bin": len(idxs),
            "positives_in_bin": pos_in_bin,
            "bin_capture_rate": capture_in_bin,
            "bin_lift": lift,
            "avg_score": float(proba_sorted[idxs].mean()) if len(idxs)>0 else np.nan
        })
    return pd.DataFrame(rows).sort_values("decile").reset_index(drop=True)

def plot_roc(y_true, y_score, title, out_path):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    auc = roc_auc_score(y_true, y_score)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(title); plt.legend(); plt.tight_layout()
    plt.savefig(out_path, dpi=160); plt.close()

def plot_pr(y_true, y_score, title, out_path):
    prec, rec, _ = precision_recall_curve(y_true, y_score)
    ap = average_precision_score(y_true, y_score)
    plt.figure()
    plt.plot(rec, prec, label=f"AP = {ap:.3f}")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(title); plt.legend(); plt.tight_layout()
    plt.savefig(out_path, dpi=160); plt.close()

def plot_lift(deciles_df, title, out_path):
    plt.figure()
    plt.plot(deciles_df["decile"], deciles_df["bin_lift"], marker="o")
    plt.xlabel("Decile (1 = highest score)"); plt.ylabel("Lift (per decile)")
    plt.title(title); plt.tight_layout()
    plt.savefig(out_path, dpi=160); plt.close()

def plot_cum_gain(deciles_df, title, out_path):
    plt.figure()
    plt.plot(deciles_df["decile"], deciles_df["bin_capture_rate"].cumsum() / deciles_df["bin_capture_rate"].sum(), marker="o")
    plt.xlabel("Decile (1 = highest score)"); plt.ylabel("Cumulative Capture Rate")
    plt.title(title); plt.tight_layout()
    plt.savefig(out_path, dpi=160); plt.close()

def plot_calibration(y_true, y_score, title, out_path, n_bins=10):
    prob_true, prob_pred = calibration_curve(y_true, y_score, n_bins=n_bins, strategy='quantile')
    plt.figure()
    plt.plot(prob_pred, prob_true, marker="o", label="Model")
    plt.plot([0,1],[0,1], linestyle="--", label="Perfect")
    plt.xlabel("Predicted probability (binned)"); plt.ylabel("Observed frequency")
    plt.title(title); plt.legend(); plt.tight_layout()
    plt.savefig(out_path, dpi=160); plt.close()

# -----------------------------------------------------------------------------
#                            EDA (core figures only)
# -----------------------------------------------------------------------------
def run_eda_and_save(df: pd.DataFrame, eda_dir: Path):
    eda_dir.mkdir(parents=True, exist_ok=True)
    dfe = df.copy()
    dfe['y_bin'] = dfe['y']

    # Figure – Target Distribution (imbalance)
    fig = plt.figure()
    counts = dfe['y_bin'].value_counts().sort_index()
    labels = ['No', 'Yes']
    plt.pie(counts, labels=labels, autopct=lambda p: f"{p:.1f}%", startangle=90)
    plt.title("Target Distribution (Imbalance)")
    fig.savefig(eda_dir / "01_target_distribution.png", dpi=150, bbox_inches="tight"); plt.close(fig)

    # Figure – % 'Unknown' by Job / Contact (heatmaps)
    for group_col, tag in [('job', 'job'), ('contact','contact')]:
        cat_cols_with_unknown = ['education', 'default', 'housing', 'loan']
        mat = {}
        for c in cat_cols_with_unknown:
            if c in dfe.columns:
                mat[c] = (dfe.assign(is_unknown=(dfe[c].astype(str).str.lower()=='unknown').astype(int))
                            .groupby(group_col)['is_unknown'].mean())
        if not mat: 
            continue
        mdf = pd.DataFrame(mat).fillna(0.0).sort_index()
        fig = plt.figure()
        plt.imshow(mdf.values, aspect='auto')
        plt.xticks(range(len(mdf.columns)), mdf.columns, rotation=45, ha='right')
        plt.yticks(range(len(mdf.index)), mdf.index)
        plt.colorbar(label="Proportion Unknown")
        plt.title(f"% 'Unknown' by {group_col.title()}")
        fig.savefig(eda_dir / f"02_unknown_heat_{tag}.png", dpi=150, bbox_inches="tight"); plt.close(fig)

    # Figure – Boxplots for campaign/previous/pdays_non999
    for col, label in [('campaign','campaign'),('previous','previous'),('pdays_non999','pdays_non999')]:
        if col in dfe.columns:
            fig = plt.figure()
            series = dfe[col].dropna()
            plt.boxplot(series.values, vert=True, showfliers=True)
            plt.title(f"Boxplot: {label}")
            plt.ylabel(col)
            fig.savefig(eda_dir / f"03_boxplot_{label}.png", dpi=150, bbox_inches="tight"); plt.close(fig)

    # Figure – Correlation matrix (numeric)
    numeric_cols = dfe.select_dtypes(include=[np.number]).columns.tolist()
    corr = dfe[numeric_cols].corr()
    fig = plt.figure()
    plt.imshow(corr.values, aspect='auto')
    plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=45, ha='right')
    plt.yticks(range(len(numeric_cols)), numeric_cols)
    plt.colorbar(label="Correlation")
    plt.title("Correlation Matrix (Numeric Features)")
    fig.savefig(eda_dir / "04_correlation_matrix.png", dpi=150, bbox_inches="tight"); plt.close(fig)

# -----------------------------------------------------------------------------
#                            Scenario: Modern Macros
# -----------------------------------------------------------------------------
def apply_modern_macros(df: pd.DataFrame,
                        euribor3m=None, cons_conf_idx=None,
                        cons_price_idx=None, emp_var_rate=None) -> pd.DataFrame:
    """
    Apply modern macroeconomic values (2023-2025) to dataset
    """
    out = df.copy()
    if euribor3m is not None and "euribor3m" in out.columns:
        out["euribor3m"] = euribor3m
    if cons_conf_idx is not None and "cons.conf.idx" in out.columns:
        out["cons.conf.idx"] = cons_conf_idx
    if cons_price_idx is not None and "cons.price.idx" in out.columns:
        out["cons.price.idx"] = cons_price_idx
    if emp_var_rate is not None and "emp.var.rate" in out.columns:
        out["emp.var.rate"] = emp_var_rate
    return out

def score_scenarios(pipe: Pipeline, X_test: pd.DataFrame, y_test: np.ndarray,
                    modern_df: pd.DataFrame, outdir: Path, tag: str):
    outdir.mkdir(parents=True, exist_ok=True)
    # Historic
    proba_hist = pipe.predict_proba(X_test)[:, 1]
    m_hist = business_point_metrics(y_test, proba_hist, top_frac=TOP_FRACTION)
    # Modern (align columns)
    modern_X_test = modern_df.loc[X_test.index, X_test.columns]
    proba_mod = pipe.predict_proba(modern_X_test)[:, 1]
    m_mod = business_point_metrics(y_test, proba_mod, top_frac=TOP_FRACTION)

    # Save comparison
    comp = pd.DataFrame({"y_true": y_test, "p_hist": proba_hist, "p_modern": proba_mod}, index=X_test.index)
    comp.to_csv(outdir / f"proba_hist_vs_modern_{tag}.csv", index=False)
    pd.DataFrame([{"scenario": "historic", **m_hist},
                  {"scenario": "modern", **m_mod}]).to_csv(outdir / f"scenario_metrics_{tag}.csv", index=False)

# -----------------------------------------------------------------------------
#                            Export Helpers
# -----------------------------------------------------------------------------
def to_csv_xlsx(df: pd.DataFrame, base_path: Path):
    base_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(base_path.with_suffix(".csv"), index=False)
    try:
        import openpyxl  # noqa: F401
        df.to_excel(base_path.with_suffix(".xlsx"), index=False)
    except Exception:
        pass  # CSV is sufficient for marking

def export_clean_and_splits(df: pd.DataFrame, X_train, X_test, data_dir: Path):
    to_csv_xlsx(df, data_dir / "cleaned_full")
    cleaned_train = df.loc[X_train.index].copy()
    cleaned_test  = df.loc[X_test.index].copy()
    to_csv_xlsx(cleaned_train, data_dir / "cleaned_train")
    to_csv_xlsx(cleaned_test,  data_dir / "cleaned_test")
    splits_idx = pd.concat([
        pd.DataFrame({"index": X_train.index, "split": "train"}),
        pd.DataFrame({"index": X_test.index,  "split": "test"})
    ]).sort_values("index").reset_index(drop=True)
    splits_idx.to_csv(data_dir / "splits_index.csv", index=False)

def export_ml_ready(pipe_lr: Pipeline, pipe_rf: Pipeline,
                    X_train, X_test, y_train, y_test, ml_dir: Path):
    ml_dir.mkdir(parents=True, exist_ok=True)

    def transform_and_save(pipe, X_tr, X_te, prefix: str):
        pre = pipe.named_steps["pre"]
        Xtr_mat = pre.transform(X_tr)
        Xte_mat = pre.transform(X_te)
        try:
            feat_names = pre.get_feature_names_out()
        except Exception:
            feat_names = [f"f{i}" for i in range(Xtr_mat.shape[1])]
        Xtr_df = pd.DataFrame(Xtr_mat, index=X_tr.index, columns=feat_names)
        Xte_df = pd.DataFrame(Xte_mat, index=X_te.index, columns=feat_names)
        to_csv_xlsx(Xtr_df, ml_dir / f"{prefix}_X_train")
        to_csv_xlsx(Xte_df, ml_dir / f"{prefix}_X_test")
        with open(ml_dir / f"feature_names_{prefix}.txt", "w", encoding="utf-8") as f:
            for n in feat_names:
                f.write(str(n) + "\n")

    transform_and_save(pipe_rf, X_train, X_test, prefix="rf")  # unscaled numerics
    transform_and_save(pipe_lr, X_train, X_test, prefix="lr")  # scaled numerics

    # Targets
    pd.DataFrame({"y": y_train}, index=X_train.index).to_csv(ml_dir / "y_train.csv", index=True)
    pd.DataFrame({"y": y_test},  index=X_test.index).to_csv(ml_dir / "y_test.csv",  index=True)

# -----------------------------------------------------------------------------
#                            Train & Evaluate (one model)
# -----------------------------------------------------------------------------
def train_and_evaluate_one(
    algo: str, X_train, y_train, X_test, y_test, outdir: Path, 
    top_frac: float = TOP_FRACTION, use_resampling: str = None
) -> Pipeline:
    outdir.mkdir(parents=True, exist_ok=True)
    
    # Create appropriate pipeline
    pipe = make_pipeline(pd.DataFrame(X_train, columns=X_train.columns), algo=algo, use_resampling=use_resampling)
    
    # If XGB, compute scale_pos_weight from training split and set it before fit
    if algo == "xgb" and HAS_XGB:
        pos = int(np.asarray(y_train).sum())
        neg = len(y_train) - pos
        spw = (neg / max(1, pos))
        pipe.named_steps["clf"].set_params(scale_pos_weight=spw)


    pipe.fit(X_train, y_train)
    proba = pipe.predict_proba(X_test)[:, 1]


    # Core and business-point metrics
    metrics_thresh05 = eval_classification(y_test, proba, threshold=0.5)
    metrics_business = business_point_metrics(y_test, proba, top_frac=top_frac)

    # Decile table
    deciles = make_decile_table(y_test, proba, n_deciles=10)

    # Model tag for naming
    model_tag = algo
    if use_resampling:
        model_tag = f"{algo}_{use_resampling}"

    # Save numeric outputs
    pd.DataFrame([{
        "model": model_tag,
        **metrics_thresh05,
        **{f"business_{k}": v for k, v in metrics_business.items()}
    }]).to_csv(outdir / f"metrics_{model_tag}.csv", index=False)

    pd.DataFrame({"model": model_tag, "y_true": y_test, "y_proba": proba}).to_csv(
        outdir / f"predictions_{model_tag}.csv", index=False
    )
    deciles.to_csv(outdir / f"deciles_{model_tag}.csv", index=False)

    # Plots
    plot_roc(y_test, proba, f"{model_tag.upper()} – ROC", outdir / f"{model_tag}_roc.png")
    plot_pr(y_test, proba, f"{model_tag.upper()} – Precision-Recall", outdir / f"{model_tag}_pr.png")
    plot_lift(deciles, f"{model_tag.upper()} – Lift by Decile", outdir / f"{model_tag}_lift.png")
    
    # Cumulative capture plot
    dec_copy = deciles.copy()
    dec_copy["cum_capture_rate"] = dec_copy["bin_capture_rate"].cumsum() / dec_copy["bin_capture_rate"].sum()
    plt.figure()
    plt.plot(dec_copy["decile"], dec_copy["cum_capture_rate"], marker="o")
    plt.xlabel("Decile (1 = highest score)"); plt.ylabel("Cumulative Capture Rate")
    plt.title(f"{model_tag.upper()} – Cumulative Capture")
    plt.tight_layout(); plt.savefig(outdir / f"{model_tag}_cum_capture.png", dpi=160); plt.close()
    
    plot_calibration(y_test, proba, f"{model_tag.upper()} – Calibration", outdir / f"{model_tag}_calibration.png")

    # Console summary
    print(f"\n=== {model_tag.upper()} ===")
    print("AUC:", metrics_thresh05["auc"], "PR-AUC:", metrics_thresh05["pr_auc"])
    print(f"At {int(top_frac*100)}% calling budget: "
          f"capture={metrics_business['decile_capture_rate']:.3f}, "
          f"lift={metrics_business['decile_lift']:.2f}, "
          f"threshold={metrics_business['threshold_at_top_frac']:.4f}")

    return pipe

# -----------------------------------------------------------------------------
#                            MAIN - COMPLETE IMPLEMENTATION
# -----------------------------------------------------------------------------
def main():
    OUTDIR.mkdir(exist_ok=True, parents=True)
    (OUTDIR / "figures").mkdir(exist_ok=True, parents=True)
    (OUTDIR / "models").mkdir(exist_ok=True, parents=True)
    (OUTDIR / "eda").mkdir(exist_ok=True, parents=True)
    (OUTDIR / "data").mkdir(exist_ok=True, parents=True)
    (OUTDIR / "ml_ready").mkdir(exist_ok=True, parents=True)

    # Load raw and clean
    print("Loading and cleaning data...")
    raw = pd.read_csv(RAW_CSV_PATH, sep=";")
    df = clean_dataset(raw)

    # EDA (core figures only, as referenced in the report)
    print("Running EDA...")
    run_eda_and_save(df, OUTDIR / "eda")

    # Split
    y = df["y"].astype(int).values
    X = df.drop(columns=["y"])
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, stratify=y, random_state=RANDOM_SEED
    )

    # Export cleaned + splits (for Appendix / reproducibility)
    print("Exporting cleaned data and splits...")
    export_clean_and_splits(df, X_train, X_test, OUTDIR / "data")

    # ---------------- Models ----------------
    print("\nTraining models...")

    # Logistic Regression (baseline)
    print("\n--- Logistic Regression Models ---")
    lr_dir = OUTDIR / "models" / "lr"
    lr_pipe = train_and_evaluate_one("lr", X_train, y_train, X_test, y_test, lr_dir)

    # Logistic Regression (ROS)
    if HAS_IMBLEARN:
        lr_ros_dir = OUTDIR / "models" / "lr_ros"
        lr_ros_pipe = train_and_evaluate_one("lr", X_train, y_train, X_test, y_test, lr_ros_dir, use_resampling="ros")
    else:
        lr_ros_pipe = None

    # Random Forest (baseline)
    print("\n--- Random Forest Models ---")
    rf_dir = OUTDIR / "models" / "rf"
    rf_pipe = train_and_evaluate_one("rf", X_train, y_train, X_test, y_test, rf_dir)

    # Random Forest (ROS)
    if HAS_IMBLEARN:
        rf_ros_dir = OUTDIR / "models" / "rf_ros"
        rf_ros_pipe = train_and_evaluate_one("rf", X_train, y_train, X_test, y_test, rf_ros_dir, use_resampling="ros")
    else:
        rf_ros_pipe = None

    # XGBoost
    print("\n--- XGBoost Models ---")
    if HAS_XGB:
        xgb_dir = OUTDIR / "models" / "xgb"
        xgb_pipe = train_and_evaluate_one("xgb", X_train, y_train, X_test, y_test, xgb_dir)
    else:
        xgb_pipe = None
        print("XGBoost not available - skipping")

    # Export ML-ready matrices (LR scaled, RF unscaled), plus targets
    print("\nExporting ML-ready datasets...")
    export_ml_ready(lr_pipe, rf_pipe, X_train, X_test, y_train, y_test, OUTDIR / "ml_ready")

    # ---------------- Scenarios ----------------
    print("\nRunning scenario analysis with modern macroeconomic data...")
    modern_df = apply_modern_macros(
        df,
        euribor3m=MODERN_MACROS['euribor3m'],
        cons_conf_idx=MODERN_MACROS['cons.conf.idx'],
        cons_price_idx=MODERN_MACROS['cons.price.idx'],
        emp_var_rate=MODERN_MACROS['emp.var.rate']
    )

    # Use all model variants for scenario check
    score_scenarios(lr_pipe, X_test, y_test, modern_df, OUTDIR / "scenario_lr", tag="lr")
    score_scenarios(rf_pipe, X_test, y_test, modern_df, OUTDIR / "scenario_rf", tag="rf")

    if lr_ros_pipe is not None:
        score_scenarios(lr_ros_pipe, X_test, y_test, modern_df, OUTDIR / "scenario_lr_ros", tag="lr_ros")
    if rf_ros_pipe is not None:
        score_scenarios(rf_ros_pipe, X_test, y_test, modern_df, OUTDIR / "scenario_rf_ros", tag="rf_ros")
    if xgb_pipe is not None:
        score_scenarios(xgb_pipe, X_test, y_test, modern_df, OUTDIR / "scenario_xgb", tag="xgb")

    # ---------------- Summary ----------------
    print("\nGenerating performance summary...")
    generate_performance_summary(OUTDIR)

    print(f"\n✅ Complete! All artifacts written to: {OUTDIR.resolve()}")
    print(f"Models trained: LR, LR+ROS, RF, RF+ROS, XGBoost")
    print(f"Scenarios tested: Historic (2008-2013) vs Modern (2023-2025)")


def generate_performance_summary(outdir: Path):
    """Generate the comprehensive performance summary table from the report"""
    summary_data = []

    model_dirs = [
        ("Logistic Regression", "lr", "models/lr/metrics_lr.csv"),
        ("Logistic Regression (ROS)", "lr_ros", "models/lr_ros/metrics_lr_ros.csv"),
        ("Random Forest", "rf", "models/rf/metrics_rf.csv"),
        ("Random Forest (ROS)", "rf_ros", "models/rf/metrics_rf_ros.csv"),
        ("XGBoost", "xgb", "models/xgb/metrics_xgb.csv")
    ]

    for model_name, model_code, metric_path in model_dirs:
        full_path = outdir / metric_path
        if full_path.exists():
            try:
                metrics = pd.read_csv(full_path).iloc[0].to_dict()
                summary_data.append({
                    "Model": model_name,
                    "Dataset": "Historic",
                    "ROC-AUC": metrics.get("auc", np.nan),
                    "PR-AUC": metrics.get("pr_auc", np.nan),
                    "Capture@10%": metrics.get("business_decile_capture_rate", np.nan),
                    "Top-decile lift": metrics.get("business_decile_lift", np.nan),
                    "Precision@0.5": metrics.get("precision@0.5", np.nan),
                    "Recall@0.5": metrics.get("recall@0.5", np.nan),
                    "Brier Score": metrics.get("brier_score", np.nan)
                })
            except Exception as e:
                print(f"Warning: Could not load metrics for {model_name}: {e}")

    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(outdir / "performance_summary.csv", index=False)
        print("Performance summary saved to: performance_summary.csv")


if __name__ == "__main__":
    main()

Loading and cleaning data...
Running EDA...
Exporting cleaned data and splits...

Training models...

--- Logistic Regression Models ---

=== LR ===
AUC: 0.8009640047879617 PR-AUC: 0.4613650187381289
At 10% calling budget: capture=0.456, lift=4.56, threshold=0.7703

=== LR_ROS ===
AUC: 0.8013884794094062 PR-AUC: 0.4618251201772269
At 10% calling budget: capture=0.456, lift=4.56, threshold=0.7712

--- Random Forest Models ---

=== RF ===
AUC: 0.8027208683192603 PR-AUC: 0.4826490668581546
At 10% calling budget: capture=0.483, lift=4.83, threshold=0.5420

=== RF_ROS ===
AUC: 0.795823874357281 PR-AUC: 0.460743952335288
At 10% calling budget: capture=0.458, lift=4.58, threshold=0.5912

--- XGBoost Models ---

=== XGB ===
AUC: 0.8009905390584461 PR-AUC: 0.4787433144341429
At 10% calling budget: capture=0.482, lift=4.82, threshold=0.7317

Exporting ML-ready datasets...

Running scenario analysis with modern macroeconomic data...

Generating performance summary...
Performance summary saved to: